In [1]:
# 加载初始值
import pickle
import os
import torch

file_name = f"initial.pkl"
if not os.path.exists(file_name):
    print(f"File {file_name} not found")
    raise FileNotFoundError

with open(file_name, "rb") as file:
    initial_global_model = pickle.load(file)

initial_global_model_params = torch.cat(
    [param.reshape(1, -1) for param in initial_global_model.values()], dim=1
)  # 初始全局模型，形状为1*d, n为参与者数量，d为梯度向量维度

print(initial_global_model_params.shape)

c:\Users\BUPT426\anaconda3\envs\clip_lora\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


torch.Size([1, 2674688])


In [2]:
attack_defense_data = {
    "MR": {
        "FLAME": "FL_Backdoor_CV/2024-10-02_03-37-50/",
        "FLTRUST": "FL_Backdoor_CV/2024-10-02_21-40-11/",
        "FOOLSGOLD": "FL_Backdoor_CV/2024-10-27_16-25-41/",
        "SECFFT": "FL_Backdoor_CV/2024-10-29_22-41-06/",
    },
    "EDGE_CASE": {
        "FLAME": "FL_Backdoor_CV/2024-10-02_09-11-28/",
        "FLTRUST": "FL_Backdoor_CV/2024-10-03_03-09-36/",
        "FOOLSGOLD": "FL_Backdoor_CV/2024-10-28_00-55-46/",
        "SECFFT": "FL_Backdoor_CV/2024-10-30_04-48-52/",
    },
    "NEUROTOXIN": {
        "FLAME": "FL_Backdoor_CV/2024-10-02_12-55-32/",
        "FLTRUST": "FL_Backdoor_CV/2024-10-03_06-48-00/",
        "FOOLSGOLD": "FL_Backdoor_CV/2024-10-28_05-59-22/",
        "SECFFT": "FL_Backdoor_CV/2024-10-30_08-40-55/",
    },
}

dataset = "fmnist"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
import os
import torch

import math
import hdbscan
import pickle


def flame(trained_params, current_model_param, param_updates):
    trained_params = trained_params.to(torch.float64)
    current_model_param = current_model_param.to(torch.float64)

    # === clustering ===
    cluster = hdbscan.HDBSCAN(
        metric="cosine",
        algorithm="generic",
        min_cluster_size=50 // 2 + 1,
        min_samples=1,
        allow_single_cluster=True,
    )
    cluster.fit(trained_params)
    predict_good = []
    for i, j in enumerate(cluster.labels_):
        if j == 0:
            predict_good.append(i)
    k = len(predict_good)

    # === median clipping ===
    model_updates = trained_params[predict_good] - current_model_param
    local_norms = torch.norm(model_updates, dim=1)
    S_t = torch.median(local_norms)
    scale = S_t / local_norms
    scale = torch.where(scale > 1, torch.ones_like(scale), scale)
    model_updates = model_updates * scale.view(-1, 1)

    # === aggregating ===
    trained_params = current_model_param + model_updates
    trained_params = trained_params.sum(dim=0) / k

    # === noising ===
    delta = 1 / (50**2)
    epsilon = 10000
    lambda_ = 1 / epsilon * (math.sqrt(2 * math.log((1.25 / delta))))
    sigma = lambda_ * S_t.numpy()
    print(
        f"sigma: {sigma}; #clean models / clean models: {k} / {predict_good}, median norm: {S_t},"
    )
    trained_params.add_(torch.normal(0, sigma, size=trained_params.size()))

    # === bn ===
    global_update = dict()
    for i, (name, param) in enumerate(param_updates.items()):
        if "num_batches_tracked" in name:
            global_update[name] = (
                1 / k * param_updates[name][predict_good].sum(dim=0, keepdim=True)
            )
        elif "running_mean" in name or "running_var" in name:
            local_norms = torch.norm(param_updates[name][predict_good], dim=1)
            S_t = torch.median(local_norms)
            scale = S_t / local_norms
            scale = torch.where(scale > 1, torch.ones_like(scale), scale)
            global_update[name] = param_updates[name][predict_good] * scale.view(-1, 1)
            global_update[name] = 1 / k * global_update[name].sum(dim=0, keepdim=True)

    return trained_params.float().to(device), global_update

In [5]:
# 输出当前路径
print(os.getcwd())

# 加载模型更新


global_param = initial_global_model_params
for attack_type, defense2data in attack_defense_data.items():
    for defense_type, defense_data in defense2data.items():
        if defense_type != "FLAME":
            continue
        for i in range(25):
            file_name = f"{defense_data}model_updates/{dataset}_{attack_type}_{i}.pkl"
            if not os.path.exists(file_name):
                print(f"File {file_name} not found")
                raise FileNotFoundError
            
            with open(file_name, "rb") as file:
                model_updates = pickle.load(file)

            # updates的形状为n*d, n为参与者数量，d为梯度向量维度
            updates = torch.cat(
                list(model_updates.values()), dim=1
            )

            # 计算每个用户的训练参数
            user_params = global_param + updates

            # 计算新的全局模型
            global_param, global_update = flame(
                user_params.cpu(), global_param.cpu(), model_updates
            )

            



c:\Users\BUPT426\Desktop\RoseAgg\RoseAgg_Latest


c:\Users\BUPT426\anaconda3\envs\clip_lora\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


sigma: 0.0005022467632193615; #clean models / clean models: 26 / [21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49], median norm: 1.2519299604304135,


c:\Users\BUPT426\anaconda3\envs\clip_lora\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


sigma: 0.0004811260317863026; #clean models / clean models: 26 / [20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 42, 43, 44, 45, 47, 48, 49], median norm: 1.1992831772082344,


...

sigma: 0.0004087413505668777; #clean models / clean models: 26 / [20, 21, 22, 23, 24, 26, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49], median norm: 1.0188528434935236,


c:\Users\BUPT426\anaconda3\envs\clip_lora\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


sigma: 0.0003961614169678062; #clean models / clean models: 26 / [20, 21, 22, 23, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49], median norm: 0.9874953576394553,


c:\Users\BUPT426\anaconda3\envs\clip_lora\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


KeyboardInterrupt: 